# Image Standards in Medicine



In [ ]:
# uncomment if you need to install dminteract
!python -m pip install -U git+https://github.com/chapmanbe/dminteract#egg=dminteract
!python -m pip install altair

In [ ]:
from dminteract.modules.m4c import *
from ipywidgets import interact, fixed
import warnings
from skimage.exposure import equalize_hist
import os
DATADIR = os.path.join("..", "DATA")
warnings.filterwarnings('ignore')

## [Digital Imaging and Communications in Medicine (DICOM)](https://en.wikipedia.org/wiki/DICOM)


>"This is rather confusing." ([Introduction to DICOM](https://nipy.org/nibabel/dicom/dicom_intro.html))

DICOM is the most important image standard in medicine. It dates back to the mid 1980s and came into wide-spread use in the 1990s and 2000s when picture archiving systems (PACS) came into wide spread use. **It can be rather confusing!**

DICOM is primarily a **communication** standard, how two computers can exchange imagine data. What we are going to be exploring here is the data representation standard.

## [DICOM defines a file format](http://dicom.nema.org/dicom/2013/output/chtml/part10/chapter_7.html)


## [And a DICOM  Dictionary](http://dicom.nema.org/dicom/2006/06_06pu.pdf)

### Which will of the most interest to us

Here are some small excerpts from the DICOM header for one of my images:

```
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'OTHER']
(0008, 0018) SOP Instance UID                    UI: 1.2.840.113619.2.312.3596.3051310.12162.1328474860.590
(0008, 0020) Study Date                          DA: '20120206'
```
-------------------------

```
(0009, 1002) [Suite id]                          UN: b'UCSD'
(0009, 1004) [Product id]                        UN: b'SIGNA '
(0009, 1030) [Service id]                        UN: b'858657MR1       '
```
-------------------------

```
(0043, 1098) [ASSET Acquisition Calibration Seri UN: Array of 54 elements
(0043, 109a) [Rx Stack Identification]           UN: b'1 '
(7fe0, 0010) Pixel Data                          OW: Array of 524288 elements
```

Let's try to analyze these. We see that each line consists of four elements:

1. **(0008, 0005)**: This is a two-tuple of 4-digit hexadecimal numbers (note: `7fe0`).
    1. The first number is the **group number**
    1. The second number is the **element number**
1. **Specific Character Set**: This is the name (human readable label for the concept).            
1. **CS**: This is the **value representation**, that is, how the value is represented, in this case a character string
1. **'ISO_IR 100'**: This is the actual value.

DICOM also defines a **value multiplicity** which does not seem to be  grabbed by the DICOM program I'm using.

As you browse through a DICOM dictionary, you will see that similar concepts tend to be collected in the same group (same group number). This implies that group numbers have semantic meaning, a violation of good standards design (see slide 22 in Omar Bouhaddou's presentation).

However, this is an artifact of the age of the DICOM standard. Since DICOM 3.0 (1993), group numbers do not have semantic meaning:

>Although similar or related Data Elements often have the same Group Number; a Data Group does not convey any semantic meaning beginning with DICOM Version 3.0. (NEMA ["The Data Set"](http://dicom.nema.org/dicom/2013/output/chtml/part05/chapter_7.html))

Odd numbered groups are **private groups.** The motivation for private groups is

>Implementations may require communication of information that cannot be contained in Standard Data Elements. Private Data Elements are intended to be used to contain such information. Such Private Data Elements shall not change the semantics of the Information Object Definition or SOP Class Definition. (NEMA ["7.8 Private Data Elements"](http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_7.8.html)

You can see in the snippets above that group `0009` is being used by the institution (UCSD) to describe itself and the machine being used to generate the images.

Private tags can be problematic as manufacturers can put all sorts of unexpected information in private tags. As an example, consider de-identification. If we wish to share images with others for research purposes, we need to remove any protected information about the patient first (e.g. patient name).

I used the [Horos](https://horosproject.org/) software package to de-identify the images before using them for this course.

In the original file, my name was present in the expected `(0010,0010)` field (as rendered by Horos):

```
PatientsName (0010,0010) CHAPMAN^BRIAN^E^
```

After de-identifcation, this field is properly blanked out (as rendered by [pydicom](https://pydicom.github.io/)):

```
(0010, 0010) Patient's Name                      PN: ''
```

However, the institution had put my name in a private field that was missed by the Horos de-identification algorithm (as rendered by pydicom):

```
(0033, 1013) [Patient's Name]                    UN: b'CHAPMAN^BRIAN^E^'
```

A more aggressive de-identification strategy might be to eliminate all private tags, but this is often problematic for images acquired with new, cutting edge technologies.

```
(0032, 1030) Reason for Study                    LO: "Clinical History:->new onset Trigeminal neuralgia - Lt sided Last creatinine:  CREAT     1.45   1/31/2012 Last BUN:  BUN       23   1/31/2012 Last GFR:  GFRNON       53   1/31/2012 Last GFR:  No results found for this basename: GFRAA Who To Call Providers (at the time that the order was placed):   Reason:->q. Special - Trigeminal Nerve Protocol Contrast:->With *** Cr at baseline, has had contrast in the past Defer to Radiologist's Protocol for Final Order?->Yes MR Angiography:->Unspecified ICD9-CM Diagnostic code : 350.1"
```
-----------
```
(0010, 21b0) Additional Patient History          LT: "Clinical History:->new onset Trigeminal neuralgia - Lt sided Last creatinine:  CREAT     1.45   1/31/2012 Last BUN:  BUN       23   1/31/2012 Last GFR:  GFRNON       53   1/31/2012 Last GFR:  No results found for this basename: GFRAA Who To Call Providers (at the time that the order was placed):   Reason:->q. Special - Trigeminal Nerve Protocol Contrast:->With *** Cr at baseline, has had contrast in the past Defer to Radiologist's Protocol for Final Order?->Yes MR Angiography:->Unspecified ICD9-CM Diagnostic code : 350.1"
```

### On whom was the image acquired?

Group `0010` has multiple elements describing the patient (me)---in this case they have been de-identified---including my non-existent mulitary rank!

```
(0010, 0010) Patient's Name                      PN: ''
(0010, 0020) Patient ID                          LO: ''
(0010, 0030) Patient's Birth Date                DA: ''
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: ''
(0010, 1030) Patient's Weight                    DS: "90.718"
(0010, 1080) Military Rank                       LO: 'ROUTINE'
```

Here is the similar section from a mammography image:

```
(0010, 0010) Patient's Name                      PN: 'Case1'
(0010, 0020) Patient ID                          LO: 'Case1'
(0010, 0030) Patient's Birth Date                DA: ''
(0010, 0040) Patient's Sex                       CS: ''
```

Notice that weight is recorded for the MRI but not the mammogram. Patient weight is an important parameter in MRI for estimating energy deposition in the subject. Weight is not important for mammography.

### How was the image acquired

Here is an excerpt from my MRI

```
(0018, 0020) Scanning Sequence                   CS: 'GR'
(0018, 0021) Sequence Variant                    CS: 'SS'
(0018, 0022) Scan Options                        CS: ['FAST_GEMS', 'ACC_GEMS', 'PFF']
(0018, 0023) MR Acquisition Type                 CS: '3D'
(0018, 0025) Angio Flag                          CS: 'N'
(0018, 0050) Slice Thickness                     DS: "1.4"
(0018, 0080) Repetition Time                     DS: "6.573"
(0018, 0081) Echo Time                           DS: "2.2"
(0018, 0082) Inversion Time                      DS: "0.0"
(0018, 0083) Number of Averages                  DS: "1.0"
(0018, 0084) Imaging Frequency                   DS: "127.71572"
(0018, 0085) Imaged Nucleus                      SH: '1H'
(0018, 0086) Echo Number(s)                      IS: "1"
(0018, 0087) Magnetic Field Strength             DS: "3.0"
(0018, 0088) Spacing Between Slices              DS: "0.7"
(0018, 0091) Echo Train Length                   IS: "1"
(0018, 0093) Percent Sampling                    DS: "100.0"
(0018, 0094) Percent Phase Field of View         DS: "100.0"
(0018, 0095) Pixel Bandwidth                     DS: "325.508"
(0018, 1000) Device Serial Number                LO: '0000000858657MR1'
(0018, 1020) Software Versions                   LO: ['24', 'LX', 'MR Software release:HD16.0_V02_1131.a']
(0018, 1030) Protocol Name                       LO: 'Alksne Trigeminal/3'
(0018, 1088) Heart Rate                          IS: "0"
(0018, 1090) Cardiac Number of Images            IS: "0"
(0018, 1094) Trigger Window                      IS: "0"
(0018, 1100) Reconstruction Diameter             DS: "200.0"
(0018, 1250) Receive Coil Name                   SH: '8HRBRAIN'
(0018, 1310) Acquisition Matrix                  US: [0, 512, 512, 0]
(0018, 1312) In-plane Phase Encoding Direction   CS: 'ROW'
(0018, 1314) Flip Angle                          DS: "40.0"
(0018, 1315) Variable Flip Angle Flag            CS: 'N'
```

Most of these are data that probably only make sense to an MR physicist. For example, `GR` stands for "gradient recall." Perhaps problematically from a standards point of view, the values do not have units of measurement. In the field

```
(0018, 0087) Magnetic Field Strength             DS: "3.0"
```
3.0 is 3.0 T (three Tesla) and 

```
(0018, 0088) Spacing Between Slices              DS: "0.7"
```
is 0.7 mm.

```
(0018, 1316) SAR                                 DS: "1.16854"
```

Some of the instantiated fields do not have meaning for this particular sequence. For example, 

```
(0018, 0082) Inversion Time                      DS: "0.0"
```

Here are entries for a mammogram:

```
(0018, 1030) Protocol Name                       LO: 'L MLO'
(0018, 1110) Distance Source to Detector         DS: "700.0"
(0018, 1111) Distance Source to Patient          DS: "620.0"
(0018, 1114) Estimated Radiographic Magnificatio DS: "1.073"
(0018, 1150) Exposure Time                       IS: "306"
(0018, 1151) X-Ray Tube Current                  IS: "180"
(0018, 1152) Exposure                            IS: "60"
(0018, 1153) Exposure in uAs                     IS: "60000"
(0018, 1166) Grid                                CS: 'NONE'
(0018, 1190) Focal Spot(s)                       DS: "0.3"
(0018, 1191) Anode Target Material               CS: 'TUNGSTEN'
(0018, 11a0) Body Part Thickness                 DS: "55.0"
(0018, 11a2) Compression Force                   DS: "139.6735"
(0018, 1200) Date of Last Calibration            DA: '19990101'
(0018, 1201) Time of Last Calibration            TM: '000000.000'
(0018, 1405) Relative X-Ray Exposure             IS: "494"
(0018, 1508) Positioner Type                     CS: 'MAMMOGRAPHIC'
(0018, 1510) Positioner Primary Angle            DS: "45.1"
(0018, 5101) View Position                       CS: 'MLO'
(0018, 7000) Detector Conditions Nominal Flag    CS: 'YES'
(0018, 7001) Detector Temperature                DS: "30.18"
(0018, 7004) Detector Type                       CS: 'DIRECT'
(0018, 700a) Detector ID                         SH: 'DET00000'
(0018, 700c) Date of Last Detector Calibration   DA: '19990101'
(0018, 700e) Time of Last Detector Calibration   TM: '000000.000'
(0018, 701a) Detector Binning                    DS: [2, 2]
(0018, 7030) Field of View Origin                DS: [0, 508]
(0018, 7032) Field of View Rotation              DS: "0.0"
(0018, 7034) Field of View Horizontal Flip       CS: 'NO'
(0018, 7050) Filter Material                     CS: 'ALUMINUM'
(0018, 7052) Filter Thickness Minimum            DS: "0.7"
(0018, 7054) Filter Thickness Maximum            DS: "0.7"
(0018, 7060) Exposure Control Mode               CS: 'AUTOMATIC'
(0018, 7062) Exposure Control Mode Description   LT: 'AutoFilter'
(0018, 8150) Exposure Time in uS                 DS: "305556.0"``
```

Here we see again that imaging parameters are lumped in group `0018` (no semantic meaning!), but they are a completely different set of element numbers. Some things to point out

* Sometimes we have explicit units ("Exposure Time in uS" micro seconds, $\mu s$), but usually not
* Lots of values are free text entries, not coded values ("ALUMINUM")

### Important Observation!

All DICOM data elements do not need to be present in the header (it would be a gigantic mess if they were). In fact, DICOM defines what is required for the header based on what kind of image is being represented. A modality like MRI is a composite of various required groups that in total represent an **information entity**.

### Exploring DICOM Images

In this section we are going to read in four distinct DICOM images. We will use these images to gain some familiarity with DICOM metadata. 

In [ ]:
img1 = pydicom.dcmread(os.path.join(DATADIR,"I1.dcm"))
img2 = pydicom.dcmread(os.path.join(DATADIR,"I2.dcm"))
img3 = pydicom.dcmread(os.path.join(DATADIR,"I3.dcm"))
img4 = pydicom.dcmread(os.path.join(DATADIR,"I4.dcm"))


### Exploration approach 1

Recalling "Concept-Based Representation" we have

* The concept (unit of thought)
* A non-semantic identifier
* A linguistic term

The non-semantic identifier is the unique concept to which the linguistic term could have synonyms or be rendered in different languages, etc.

In the DICOM standard the non-semantic identifers are the group-element tuple expressed as hexadecimal integers; this is the unique value in the concept.

Let's try browsing our images by entering different group-element tuples

* Select different images with the drop down menu.
* Type in different integers in the group and elem fields

In [ ]:
_= interact(view_dicom_data, img={"img1":img1,
               "img2":img2, 
               "img3":img3, 
               "img4":img4}.items(), group="0x008", elem="0x008")

#### Unless you are really savvy with DICOM or you have a DICOM reference nearby this feels like shooting in the dark 

### Exploring by Name

In the cells below, we take a more human approach. In each cell we have a different image. You can select a different DICOM element name and then see the group-element tuple and the value.

In [ ]:
_ = interact(view_dicom_data_rev, 
             img=fixed(img1), 
             item=get_rev_dict(img1))

In [ ]:
_ = interact(view_dicom_data_rev, 
             img=fixed(img2), 
             item=get_rev_dict(img2))

In [ ]:
_ = interact(view_dicom_data_rev, 
             img=fixed(img3), 
             item=get_rev_dict(img3))

In [ ]:
_ = interact(view_dicom_data_rev, 
             img=fixed(img4), 
             item=get_rev_dict(img4))

#### Use the cells above to explore the DICOM metadata to answer the following questions

In [ ]:
for q in question_banks["qbank3"].values():
    display(q)

### Before we leave Take a look at what the images look like

In [ ]:
_=interact(lambda x:imshow(equalize_hist(x.pixel_array), cmap="gray"), 
           x={"img1":img1,
               "img2":img2, 
               "img3":img3, 
               "img4":img4}.items())

### Modality Standards

The DICOM standard defines data elements specific to each imaging modality. You can browse the standards for the imaging modalities examined here. 

* [Mammography](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.8.31.html).
* [CT](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.8.2.html)
* [MR](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.8.3.html)

### [Move onto the next notebook](dicom_intro_pixels_voxels.ipynb)